This notebook merges the merged file of 2019 LA Calls & Neighborhood Council geometry with the Crime Data from 2019 (call_NC_merge_2019.pickle)


In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import geopandas as gpd

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
inpickle = r'C:\Documents\projects\HackLA\311\data\WorkedonData\Geo\call_NC_merge_2019.pickle'

In [5]:
callNCGeo = pd.read_pickle(inpickle)
callNCGeo.head(2)

,OBJECTID,NCName,NC_ID,CERTIFIED,TOOLTIP,SERVICE_RE,geometry,SRNumber,CreatDt,UpdateDt,ReqTpe,ReqSrc,Address,ZipCode,Lat,Lon,Location,Area
0,1.0,ARLETA NC,6,2002-10-22,ARLETA NC,REGION 1 - NORTH EAST VALLEY,"POLYGON ((-118.44305 34.26328, -118.44303 34.2...",1-1262725561,01/01/2019 10:30:00 AM,01/05/2019 11:16:00 AM,Bulky Items,Call,"9489 N DORRINGTON AVE, 91331",91331.0,34.242677,-118.444364,"(34.2426768839, -118.444363615)",MISSION
1,1.0,ARLETA NC,6,2002-10-22,ARLETA NC,REGION 1 - NORTH EAST VALLEY,"POLYGON ((-118.44305 34.26328, -118.44303 34.2...",1-1262780112,01/01/2019 02:46:00 PM,01/02/2019 08:09:00 AM,Graffiti Removal,Mobile App,"14222 W WEIDNER ST, 91331",91331.0,34.258201,-118.442768,"(34.258200559, -118.442768159)",MISSION


In [6]:
crime_data = pd.read_csv(r'C:\Documents\projects\HackLA\311\data\crime data\Crime_Data_from_2010_to_2019.csv', index_col=None)

In [7]:
crime_data.head(2)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962


In [8]:
# remove the unnecessary columns
crime_data=crime_data[['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA NAME', 'Rpt Dist No', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Desc', 'Weapon Used Cd', 
'Weapon Desc', 'Status Desc','LOCATION', 'LAT', 'LON']]

In [9]:
crime_data.head(2)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Rpt Dist No,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status Desc,LOCATION,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,Newton,1385,48,M,H,SINGLE FAMILY DWELLING,NaN,NaN,Adult Arrest,300 E GAGE AV,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,Pacific,1485,0,M,W,STREET,NaN,NaN,Invest Cont,SEPULVEDA BL,33.9599,-118.3962


In [10]:
# There are 2 date columns in the crime_data df (dataframe). Change the column type to datetype

crime_data['DATE OCC']= pd.to_datetime(crime_data['DATE OCC'])

crime_data['Date Rptd']= pd.to_datetime(crime_data['Date Rptd'])

Create df of crime data from only 2019

In [11]:
crime_data_2019 = crime_data[crime_data['Date Rptd'].dt.year==2019]

In [12]:
crime_data_2019.head(2)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Rpt Dist No,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status Desc,LOCATION,LAT,LON
85876,190705631,2019-02-04,2010-07-01,1100,Wilshire,773,60,F,B,SINGLE FAMILY DWELLING,NaN,NaN,Invest Cont,1900 S RIDGELEY DR,34.0409,-118.3586
86769,191300738,2019-06-11,2010-06-07,1800,Newton,1383,23,M,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,Invest Cont,5800 S BROADWAY,33.9829,-118.2783


In [13]:
crime_data_2019.shape

(220556, 16)

In [14]:
crime_data_2019['Date Rptd'].min()

Timestamp('2019-01-01 00:00:00')

In [15]:
crime_data_2019['Date Rptd'].max()

Timestamp('2019-12-31 00:00:00')

In [16]:
crime_data_2019['DATE OCC'].min()

Timestamp('2010-01-01 00:00:00')

In [17]:
crime_data_2019['DATE OCC'].max()

Timestamp('2019-12-31 00:00:00')

In [18]:
dups = crime_data_2019[crime_data_2019.duplicated()]

In [19]:
dups.shape

(0, 16)

No dupes

In [20]:
crime_data_2019.isnull().sum()

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA NAME              0
Rpt Dist No            0
Vict Age               0
Vict Sex           19922
Vict Descent       19926
Premis Desc           91
Weapon Used Cd    141939
Weapon Desc       141939
Status Desc            0
LOCATION               0
LAT                    0
LON                    0
dtype: int64

In [21]:
crime_data_2019.head(2)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Rpt Dist No,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status Desc,LOCATION,LAT,LON
85876,190705631,2019-02-04,2010-07-01,1100,Wilshire,773,60,F,B,SINGLE FAMILY DWELLING,NaN,NaN,Invest Cont,1900 S RIDGELEY DR,34.0409,-118.3586
86769,191300738,2019-06-11,2010-06-07,1800,Newton,1383,23,M,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,Invest Cont,5800 S BROADWAY,33.9829,-118.2783


In [22]:
callNCGeo.head(2)

,OBJECTID,NCName,NC_ID,CERTIFIED,TOOLTIP,SERVICE_RE,geometry,SRNumber,CreatDt,UpdateDt,ReqTpe,ReqSrc,Address,ZipCode,Lat,Lon,Location,Area
0,1.0,ARLETA NC,6,2002-10-22,ARLETA NC,REGION 1 - NORTH EAST VALLEY,"POLYGON ((-118.44305 34.26328, -118.44303 34.2...",1-1262725561,01/01/2019 10:30:00 AM,01/05/2019 11:16:00 AM,Bulky Items,Call,"9489 N DORRINGTON AVE, 91331",91331.0,34.242677,-118.444364,"(34.2426768839, -118.444363615)",MISSION
1,1.0,ARLETA NC,6,2002-10-22,ARLETA NC,REGION 1 - NORTH EAST VALLEY,"POLYGON ((-118.44305 34.26328, -118.44303 34.2...",1-1262780112,01/01/2019 02:46:00 PM,01/02/2019 08:09:00 AM,Graffiti Removal,Mobile App,"14222 W WEIDNER ST, 91331",91331.0,34.258201,-118.442768,"(34.258200559, -118.442768159)",MISSION


In [23]:
#create geometry column for crime - turn it into geopandas frame
gdfCrime = gpd.GeoDataFrame(crime_data_2019, geometry=gpd.points_from_xy(crime_data_2019['LON'], crime_data_2019['LAT']))

In [24]:
gdfCrime.head(2)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Rpt Dist No,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status Desc,LOCATION,LAT,LON,geometry
85876,190705631,2019-02-04,2010-07-01,1100,Wilshire,773,60,F,B,SINGLE FAMILY DWELLING,NaN,NaN,Invest Cont,1900 S RIDGELEY DR,34.0409,-118.3586,POINT (-118.35860 34.04090)
86769,191300738,2019-06-11,2010-06-07,1800,Newton,1383,23,M,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,Invest Cont,5800 S BROADWAY,33.9829,-118.2783,POINT (-118.27830 33.98290)


In [51]:
gdfCrime['geometry'] = gdfCrime.geometry.buffer(0.01)

C:\Users\lizge\AppData\Local\Temp\ipykernel_12248\1144187549.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdfCrime['geometry'] = gdfCrime.geometry.buffer(0.01)


In [26]:
gdfCrime.head(2)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Rpt Dist No,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status Desc,LOCATION,LAT,LON,geometry
85876,190705631,2019-02-04,2010-07-01,1100,Wilshire,773,60,F,B,SINGLE FAMILY DWELLING,NaN,NaN,Invest Cont,1900 S RIDGELEY DR,34.0409,-118.3586,"POLYGON ((-118.35760 34.04090, -118.35760 34.0..."
86769,191300738,2019-06-11,2010-06-07,1800,Newton,1383,23,M,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,Invest Cont,5800 S BROADWAY,33.9829,-118.2783,"POLYGON ((-118.27730 33.98290, -118.27730 33.9..."


In [31]:
#change Date columns back to string format because shp and json files don't take dates
gdfCrime['DATE OCC'] = gdfCrime['DATE OCC'].astype(str)

In [32]:
gdfCrime['Date Rptd'] = gdfCrime['Date Rptd'].astype(str)

In [33]:
#save gdfCrime to hard drive 
gdfCrime.to_file(r'C:\Documents\projects\HackLA\311\data\WorkedonData\Geo\crime.crimeGeo.shp')

C:\Users\lizge\AppData\Local\Temp\ipykernel_12248\1006562327.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdfCrime.to_file(r'C:\Documents\projects\HackLA\311\data\WorkedonData\Geo\crime.crimeGeo.shp')


In [39]:
callNCGeo.head(2)

,OBJECTID,NCName,NC_ID,CERTIFIED,TOOLTIP,SERVICE_RE,geometry,SRNumber,CreatDt,UpdateDt,ReqTpe,ReqSrc,Address,ZipCode,Lat,Lon,Location,Area
0,1.0,ARLETA NC,6,2002-10-22,ARLETA NC,REGION 1 - NORTH EAST VALLEY,"POLYGON ((-118.44305 34.26328, -118.44303 34.2...",1-1262725561,01/01/2019 10:30:00 AM,01/05/2019 11:16:00 AM,Bulky Items,Call,"9489 N DORRINGTON AVE, 91331",91331.0,34.242677,-118.444364,"(34.2426768839, -118.444363615)",MISSION
1,1.0,ARLETA NC,6,2002-10-22,ARLETA NC,REGION 1 - NORTH EAST VALLEY,"POLYGON ((-118.44305 34.26328, -118.44303 34.2...",1-1262780112,01/01/2019 02:46:00 PM,01/02/2019 08:09:00 AM,Graffiti Removal,Mobile App,"14222 W WEIDNER ST, 91331",91331.0,34.258201,-118.442768,"(34.258200559, -118.442768159)",MISSION


In [42]:
callNCGeo['CreatDt'] = callNCGeo['CreatDt'].astype(str)

In [43]:
callNCGeo['UpdateDt'] = callNCGeo['UpdateDt'].astype(str)


In [50]:
gdfCrime.crs = "EPSG:4326"

In [44]:
callNCGeo.shape

(1288803, 18)

In [45]:
gdfCrime.shape

(220556, 17)

In [52]:
callNCGeo.sjoin(gdfCrime, how="left")

KeyboardInterrupt: 